# **SAP Solutions Intent Score Calculation Task**

## **Aim**
To create a comprehensive analysis of a company&#39;s likelihood to require specific SAP
solutions. This analysis will help the team to better understand the potential client needs and tailor the
approach accordingly.

###### **SAP Solutions to focus on are**

1. SAP S/4HANA
2. SAP SuccessFactors
3. SAP Ariba

### **Indicators List**

The list of indicators that we are using to suggest a SAP solution to a company are

1. **Company Name**: [Company Name]
2. **Industry** : [Industry Type]
3. **Company Size**: [Number of Employees]
4. **Annual Revenue**: [Annual Revenue]
5. **Global Presence**: [Yes/No]
6. **Existing ERP Systems**: [Yes/No/Details]
7. **HR Department Size**: [Number of Employees]
8. **Current HR Software**: [Yes/No/Details]
9. **Employee Turnover Rate**: [Percentage]
10. **Growth Rate**: [Percentage]
11. **Size of Procurement Department**: [Number of Employees]
12. **Number of Suppliers**: [Number]
13. **Annual Procurement Spend**: [Amount]
14. **Complexity of Supply Chain**: [Low/Medium/High]
15. **Current Procurement Systems**: [Yes/No/Details]


### **LLM Models**

1. **mistralai/Mistral-7B-Instruct-v0.2**
2. **HuggingFaceH4/zephyr-7b-beta**

## **Methodology**

The project is designed as an Initial Prompt followed by a RAG Prompt for efficient tokenization -

* Initial Prompt
  * An Initial prompt is designed to generate intent score and short reasoning for the score based on the company's profile as per the Indicators list

* RAG Prompt
  * A pdf document explaining the three SAP solutions and its uses are used in the RAG to ensure validation and reduce hallucinations.
  * The document, along with the score and reasoning generated in the initial prompt is then used in this prompt to retreive the detailed reasoning and factors affecting increase/decrease in score in a structured format  


### **Code**

### **Install required packages**

In [ ]:
!pip install huggingface-hub
!pip -q install langchain
!pip -q install langchain_community
!pip -q install pypdf
!pip -q install chromadb
!pip -q install tiktoken
!pip install langchainhub
!pip -q install langchain_community
!pip -q install langchain
!pip -q install --upgrade huggingface_hub
!pip -q install langchain_huggingface

### **Import required packages**

In [ ]:
import os
import numpy as np
import torch
from getpass import getpass
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline, AutoTokenizer, AutoModel
from google.colab import files

from langchain import hub, LLMChain
from langchain.llms import HuggingFaceHub, HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.embeddings import HuggingFaceHubEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders.csv_loader import CSVLoader


### **Provide your HuggingFace api key/access token**

In [ ]:
# Enter your HuggingFace access token when prompted

pass_token = getpass("Enter your HuggingFace access token: ")

os.environ["HF_TOKEN"] = pass_token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = pass_token

del pass_token


Enter your HuggingFace access token: ··········


**Function to get Initial Prompt result. The company's profile data (indicators list) will be entered here**

In [ ]:
def initial_prompt(model):

  llm = HuggingFaceEndpoint(
    repo_id=model,
    task="text-generation",
    max_new_tokens = 5000,
    top_k = 30,
    temperature = 0.1,
    repetition_penalty = 1.03,
)
  name=str(input("Name of the Company :"))
  industry=str(input("Industry of the Company :"))
  employees=str(input("Number of employees :"))
  revenue=str(input("Annual Revenue in Cr:"))
  erp=str(input("Existing ERP Systems :"))
  hr_emp=str(input("HR Department Size :"))
  hr_soft=str(input("Current HR Software :"))
  emp_turn_over=str(input("Employee Turnover Rate % :"))
  growth=str(input("Growth Rate % :"))
  procurement_dept=str(input("Size of Procurement Department :"))
  suppliers=str(input("Number of Suppliers :"))
  spend=str(input("Annual Procurement Spending in Cr :"))
  supply_chain=str(input("Complexity of Supply Chain (low/medium/high) :"))
  procure=str(input("Current Procurement Systems :"))

  company_profile="""
     {name} is a company belonging to {industry} industry. The company profile is as follows

      Company Size: {employees}
      Annual Revenue: {revenue} Crores
      Existing ERP Systems: {erp}
      HR Department Size: {hr_emp}
      Current HR Software: {hr_soft}
      Employee Turnover Rate: {emp_turn_over} %
      Growth Rate: {growth} %
      Size of Procurement Department: {procurement_dept}
      Number of Suppliers: {suppliers}
      Annual Procurement Spend: {spend} Crores
      Complexity of Supply Chain: {supply_chain}
      Current Procurement Systems: {procure}""".format(name=name,
      industry=industry,
      employees=employees,
      revenue=revenue,
      erp=erp,
      hr_emp=hr_emp,
      hr_soft=hr_soft,
      emp_turn_over=emp_turn_over,
      growth=growth,
      procurement_dept=procurement_dept,
      suppliers=suppliers,
      spend=spend,
      supply_chain=supply_chain,
      procure=procure)

  prompt_template = PromptTemplate.from_template(
  """You are a SAP solutions expert in SAP S/4HANA, SAP SuccessFactors and SAP Ariba.

      {company_profile}

      Generate Intent Score for the SAP solutions - SAP S/4HANA, SAP SuccessFactors, SAP Ariba:

      Intent Score: Based on company profile, provide intent score (0-10) for adopting this SAP solution. Provide only a numeric score and reasoning for the score in 5 words

      Example

      SAP  S/4HANA:
      Intent Score: 8


      SAP  SuccessFactors:
      Intent Score: 5


      SAP  Ariba:
      Intent Score: 7

  """)

  messages = prompt_template.format(company_profile=company_profile)
  chat_model = ChatHuggingFace(llm = llm)

  content=[]
  response = chat_model.invoke(messages)
  content.append(response.content)
  return company_profile,content

**Function to vectorise RAG document**

In [ ]:
def vector(doc,embedding):
  chroma_db_dir = "./chroma_vector_db"
# Create the directory if it doesn't exist
  os.makedirs(chroma_db_dir, exist_ok=True)

  !rm -rf ./docs/chroma  # remove old database files if any

  embedding = HuggingFaceHubEmbeddings()

  vectordb = Chroma.from_documents(
      documents=doc, # splits we created earlier
      embedding=embedding,
      persist_directory=chroma_db_dir # save the directory
)
  return vectordb

**Function to load, embed and vectorise document**

In [ ]:
def doc_process(path):
# Load PDF
  loaders = [
      # Duplicate documents on purpose
      PyPDFLoader(path),
  ]
  docs = []
  for loader in loaders:
      docs.extend(loader.load())

  # Split
  #from langchain.text_splitter import RecursiveCharacterTextSplitter
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = 500,
      chunk_overlap = 50
  )

  splits = text_splitter.split_documents(docs)
  embedding = HuggingFaceHubEmbeddings()

  vectordb=vector(doc=splits,embedding=embedding)

  return vectordb

**Function to get RAG Prompt results**

In [ ]:
def run_rag(model,vectordb,company_profile,content):
  chat_llm = HuggingFaceEndpoint(
    repo_id=model,
    task="text-generation",
    max_new_tokens = 1000,
    top_k = 30,
    temperature = 0.1,
    repetition_penalty = 1.03,)

  llm = ChatHuggingFace(llm=chat_llm)
  prompt = hub.pull("rlm/rag-prompt")


  # Build prompt
  template = """
  You are a SAP solutions expert in SAP S/4HANA, SAP SuccessFactors and SAP Ariba.Use the following pieces of context to answer the question at the end.
  If you don't know the answer, just say that you don't know, don't try to make up an answer.
  Always say "thanks for asking!" at the end of the answer.

  {context}
  Question: {question}
  Helpful Answer:"""
  QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)
  qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 2, "fetch_k":6} ), # "k":2, "fetch_k":3
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True
                                       )

  saps=['SAP S/4HANA','SAP SuccessFactors','SAP Ariba']
  results =[]
  for sap in saps:

    question = """
        {company_profile}

        {content}

       Do not provide any summary. Output should be provided as a bullet-point list as follows .

      For the {sap}

        * Intent Score :
        * Reasoning : Justification for the intent score (based on the company's details provided)
        * Factors Increasing Intent Score:
          - Challenges the company is facing.
          - Current systems or processes that align with the capabilities of the SAP solution.
        * Factors Decreasing Intent Score:
          - Areas where the SAP solution might not be the best fit.
          - Conflicting objectives or systems already in place.

    """.format(company_profile=company_profile,content=content,sap=sap)
    result = qa_chain.invoke({"query": question})
    results.append(result["result"])
  return results[0],results[1],results[2]

## **Results**

#### **LLM - mistralai/Mistral-7B-Instruct-v0.2**

In [ ]:
model="mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
company_profile,content=initial_prompt(model)

<ipython-input-4-09e5a8a60f50>:3: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceEndpoint(
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Name of the Company :Ask Automotive Limited
Industry of the Company :Components Manufacturing
Number of employees :7000+
Annual Revenue in Cr:2890
Existing ERP Systems :NetSuite
HR Department Size :100
Current HR Software :BambooHR, Gusto
Employee Turnover Rate % :2
Growth Rate % :40
Size of Procurement Department :200
Number of Suppliers :500
Annual Procurement Spending in Cr :300
Complexity of Supply Chain (low/medium/high) :low
Current Procurement Systems :yes


<ipython-input-4-09e5a8a60f50>:82: LangChainDeprecationWarning: The class `ChatHuggingFace` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import ChatHuggingFace``.
  chat_model = ChatHuggingFace(llm = llm)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
vectordb=doc_process("/content/SAP_doc.pdf")

In [ ]:
hana,successfactors,ariba=run_rag(model=model,vectordb=vectordb,company_profile=company_profile,content=content)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
print(hana)

 For the SAP S/4HANA:

* Intent Score: 9
* Reasoning: Large company size and high growth rate indicate potential for scalability and need for advanced ERP capabilities.
* Factors Increasing Intent Score:
  - The company's size and growth rate suggest that they may outgrow their current NetSuite ERP system and require a more robust solution like SAP S/4HANA.
  - The HR department size and employee turnover rate are not significant factors in determining the intent score since SAP S/4HANA is primarily an ERP solution.
* Factors Decreasing Intent Score:
  - There are no apparent factors decreasing the intent score based on the provided context. However, the current ERP system being NetSuite might be a consideration if there are specific integrations or customizations already in place.

Thanks for asking!


In [ ]:
print(successfactors)

 For the SAP SuccessFactors:

* Intent Score: 6
* Reasoning: The company's small HR department size and low employee turnover rate suggest that their current HR systems, BambooHR and Gusto, may be sufficient for their needs at present. However, the potential for future growth could change this.
* Factors Increasing Intent Score:
  - The company's high growth rate indicates a need for a more robust and scalable HR solution to support future expansion.
  - The company's current HR systems may not offer advanced features and capabilities required for managing a larger workforce and more complex HR processes.
* Factors Decreasing Intent Score:
  - The company's current HR systems are already in place and may not be easily replaced, making a transition to SAP SuccessFactors a more complex and time-consuming process.
  - The company's low employee turnover rate may not justify the investment in a more advanced HR solution if the current systems are meeting their needs effectively.

Thanks fo

In [ ]:
print(ariba)

 For the SAP Ariba:

* Intent Score: 9
* Reasoning: The company's significant procurement spend and large number of suppliers indicate potential for improved supply chain management and cost savings with Ariba.
* Factors Increasing Intent Score:
  - The company's high growth rate and large size suggest the need for a more robust and scalable procurement solution to manage their increasing spend and supplier base.
  - The current procurement systems are already in place, making it easier for the company to consider an integrated solution like SAP Ariba.
* Factors Decreasing Intent Score:
  - The low complexity of the supply chain might not require the full suite of capabilities offered by SAP Ariba, but it could still provide benefits in areas such as contract management and invoice processing.
  - The company's current HR and ERP systems are not SAP solutions, which could make an implementation more complex and potentially require additional integrations.

Thanks for asking!


#### **LLM - HuggingFaceH4/zephyr-7b-beta**

In [ ]:
model="HuggingFaceH4/zephyr-7b-beta"

In [ ]:
company_profile,content=initial_prompt(model)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Name of the Company :Ask Automotive Limited
Industry of the Company :Components Manufacturing
Number of employees :7000+
Annual Revenue in Cr:2890
Existing ERP Systems :NetSuite
HR Department Size :100
Current HR Software :BambooHR, Gusto
Employee Turnover Rate % :2
Growth Rate % :40
Size of Procurement Department :200
Number of Suppliers :500
Annual Procurement Spending in Cr :300
Complexity of Supply Chain (low/medium/high) :low
Current Procurement Systems :yes


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
vectordb=doc_process("/content/SAP_doc.pdf")

In [ ]:
hana,successfactors,ariba=run_rag(model=model,vectordb=vectordb,company_profile=company_profile,content=content)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
print(hana)

For the SAP S/4HANA:

* Intent Score: 9
* Reasoning: High annual revenue and complex ERP systems require a scalable and efficient solution. SAP S/4HANA's real-time analytics, simplified data management, and integration with NetSuite make it an ideal choice for Ask Automotive's growth and expansion.
* Factors Increasing Intent Score:
  - Challenges: Ask Automotive's high annual revenue and complex ERP systems require a solution that can handle large volumes of data and provide real-time insights.
  - Current systems or processes: Ask Automotive's current ERP system, NetSuite, can be integrated with SAP S/4HANA, providing a seamless transition and minimizing disruption to existing processes.
* Factors Decreasing Intent Score:
  - Areas where the SAP solution might not be the best fit: SAP S/4HANA may not be the best fit for Ask Automotive's smaller departments, such as finance or logistics, as they may not require the same level of functionality and scalability as the HR and procurement 

In [ ]:
print(successfactors)

For the SAP SuccessFactors:

* Intent Score: 6
* Reasoning: While the HR department is small, the high employee turnover rate and rapid growth indicate a need for a comprehensive HR solution. SAP SuccessFactors' talent management, learning, and performance capabilities can help Ask Automotive attract, develop, and retain top talent.
* Factors Increasing Intent Score:
  - High employee turnover rate: SAP SuccessFactors' talent management and performance capabilities can help Ask Automotive identify and address performance issues, as well as provide opportunities for career development and succession planning. This can help reduce turnover and improve employee engagement and satisfaction.
  - Rapid growth: SAP SuccessFactors' learning and development capabilities can help Ask Automotive provide the necessary training and development programs to support the company's growth and expansion. This can also help attract and retain top talent in a competitive market.
* Factors Decreasing Intent

In [ ]:
print(ariba)

For the SAP Ariba:

* Intent Score: 8
* Reasoning: The large procurement department and high annual spend require a centralized and automated procurement system. SAP Ariba's supplier management, e-procurement, and spend analysis features can help Ask Automotive streamline processes, reduce costs, and improve supplier relationships.
* Factors Increasing Intent Score:
  - Large procurement department and high annual spend indicate a need for a comprehensive and efficient procurement solution.
  - Current procurement systems may not be able to handle the increasing volume and complexity of transactions, making SAP Ariba's automation and centralization capabilities more attractive.
* Factors Decreasing Intent Score:
  - The low complexity of the supply chain may not require the full range of SAP Ariba's capabilities, such as supply chain planning and collaboration.
  - Ask Automotive may have existing relationships with suppliers that are not compatible with SAP Ariba's supplier management

## **Conclusion**
 * **SAP S/4HANA** is given an intent score 9 by both LLM models
 * **SAP SuccessFactors** is given an intent score 6 by both LLM models
 * **SAP Ariba** is given an intent score 9 by Mistral and score 8 by zephyr-7b-beta. The scores are not consistent across models

 Based on the above, SAP S\4HANA is a good SAP solution to implement for Ask Automotive Limited